In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os

# Define the repository URL and the local directory where you want to clone/pull the repository
repo_url = 'https://github.com/aakashvardhan/s11-gradcam.git'
local_dir = '/content/s11-gradcam'



# Check if the local directory already exists
if not os.path.exists(local_dir):
    # Clone the repository because it does not exist
    !git clone {repo_url}
else:
    # Change directory to the local repository
    %cd {local_dir}
    # Pull the latest changes because the repository already exists
    !git pull

In [ ]:
import sys
sys.path.append('/content/s11-gradcam')
sys.path.append('/content/s11-gradcam/models')
sys.path.append('/content/s11-gradcam/utils')

In [ ]:
!pip install torch-lr-finder

In [ ]:
from config import get_config
config = get_config()
from main import main,set_seeds,setup_cifar10
import torch
from torch_lr_finder import LRFinder
from models.resnet import ResNet18
from models.model_utils import model_summary, adam_optimizer
import torch.nn as nn
from trainer import ModelTrainer
from torch.optim.lr_scheduler import OneCycleLR
set_seeds()

In [ ]:
config['epochs'] = 30
config['lr_scheduler'] = 'one_cycle'
config['lr'] = 0.001

In [ ]:
train_data, test_data, train_loader, test_loader = setup_cifar10(config)

In [ ]:
ocp_scheduler = OneCycleLR(
        optimizer,
        max_lr=config["max_lr"],
        steps_per_epoch=len(train_loader),
        epochs=config["epochs"],
        pct_start=5 / config["epochs"],
        div_factor=100,
        three_phase=False,
        final_div_factor=100,
        anneal_strategy="linear")

In [ ]:
criterion = nn.CrossEntropyLoss()

In [ ]:
# Create an instance of the ModelTrainer class
trainer = ModelTrainer(model, device, train_loader, test_loader, optimizer, criterion, ocp_scheduler)

In [ ]:
model = ResNet18()
model_summary(model, input_size=(3, 32, 32))